# Capital One Airline Data Challenge — Q1 2019

This notebook orchestrates the G-A-M-E workflow end-to-end using utilities in `src/`:
- Ingestion with explicit schema and Q1-2019 scope
- Data quality checks and issue logging
- Feature engineering (route IDs, on-time flags)
- Busiest routes, revenue/profit, integrated summary
- Recommendations and breakeven sizing
- Chart generation saved to `docs/`


In [1]:
# Ensure project root on sys.path and show pandas version
import os, sys
proj_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)
import pandas as pd
print(sys.executable, pd.__version__)
print("Using project root:", proj_root)
print("pandas:", pd.__version__)


/Users/chi-way/Desktop/Yale SOM/Full-time/CapitalOne/DA-Airline-Data-Challenge-main/.venv/bin/python 2.3.3
Using project root: /Users/chi-way/Desktop/Yale SOM/Full-time/CapitalOne/DA-Airline-Data-Challenge-main
pandas: 2.3.3


In [2]:
# Preamble: imports and paths
import os
import pandas as pd
from src.ingest import ingest_airports, ingest_flights, ingest_tickets, write_processed
from src.quality import (
    check_nulls,
    check_duplicates,
    check_iata_format,
    check_cancelled_binary,
    check_occupancy_bounds,
)
from src.features import build_round_trip_id, build_leg_direction, on_time_flags, mask_cancelled

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname("."), os.pardir))
DATA = os.path.join(PROJECT_ROOT, 'data')
RAW = os.path.join(DATA, 'raw')
PROC = os.path.join(DATA, 'processed')
DOCS = os.path.join(PROJECT_ROOT, 'docs')
META = os.path.join(DATA, 'Airline_Challenge_Metadata.xlsx')
os.makedirs(PROC, exist_ok=True)
os.makedirs(DOCS, exist_ok=True)


In [3]:
# Ingestion (Q1 2019)
airports = ingest_airports(os.path.join(RAW,'Airport_Codes.csv'), META)
flights = ingest_flights(os.path.join(RAW,'Flights.csv'), META)
tickets = ingest_tickets(os.path.join(RAW,'Tickets.csv'), META)
write_processed(airports, os.path.join(PROC,'airports.csv'))
write_processed(flights, os.path.join(PROC,'flights.csv'))
write_processed(tickets, os.path.join(PROC,'tickets.csv'))
len(airports), len(flights), len(tickets)


(55369, 1910836, 1167285)

In [4]:
# Data quality checks
issues = []
issues.append(check_nulls(flights, []))
issues.append(check_duplicates(airports, ['IATA_CODE']))
if 'IATA_CODE' in airports.columns:
    issues.append(check_iata_format(airports['IATA_CODE']))
if 'CANCELLED' in flights.columns:
    issues.append(check_cancelled_binary(flights['CANCELLED']))
if 'OCCUPANCY_RATE' in flights.columns:
    issues.append(check_occupancy_bounds(flights['OCCUPANCY_RATE']))
issue_log = pd.concat([i for i in issues if i is not None and not i.empty], ignore_index=True) if any([i is not None and not i.empty for i in issues]) else pd.DataFrame()
issue_log.to_csv(os.path.join(DOCS,'issue_log.csv'), index=False) if not issue_log.empty else None
issue_log.head() if not issue_log.empty else 'No issues detected at selected checks.'


,column,issue,count,IATA_CODE,value
0,TAIL_NUM,nulls,12106.0,<NA>,<NA>
1,DEP_DELAY,nulls,50201.0,<NA>,<NA>
2,ARR_DELAY,nulls,55831.0,<NA>,<NA>
3,AIR_TIME,nulls,55831.0,<NA>,<NA>
4,NaN,duplicate_key,NaN,NAN,<NA>


In [5]:
# Feature engineering
flights_eng = flights.copy()
flights_eng['round_trip_id'] = build_round_trip_id(flights_eng)
flights_eng['leg_dir'] = build_leg_direction(flights_eng)
ot = on_time_flags(flights_eng)
flights_eng = pd.concat([flights_eng, ot], axis=1)
valid_leg = mask_cancelled(flights_eng)
write_processed(flights_eng, os.path.join(PROC,'flights_engineered.csv'))
flights_eng.head(3)


/Users/chi-way/Desktop/Yale SOM/Full-time/CapitalOne/DA-Airline-Data-Challenge-main/src/features.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sorted_pair = pair.apply(lambda r: "-".join(sorted([r[0], r[1]])), axis=1)


,FL_DATE,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,...,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,OCCUPANCY_RATE,round_trip_id,leg_dir,on_time_dep,on_time_arr,on_time_both
0,2019-03-02,WN,N955WN,4591,14635,RSW,"Fort Myers, FL",11042,CLE,"Cleveland, OH",...,-6.0,0,143.0,1025.0,0.97,CLE-RSW,RSW>CLE,True,True,True
1,2019-03-02,WN,N8686A,3231,14635,RSW,"Fort Myers, FL",11066,CMH,"Columbus, OH",...,5.0,0,135.0,930.0,0.55,CMH-RSW,RSW>CMH,False,False,False
2,2019-03-02,WN,N201LV,3383,14635,RSW,"Fort Myers, FL",11066,CMH,"Columbus, OH",...,4.0,0,132.0,930.0,0.91,CMH-RSW,RSW>CMH,True,False,False


In [6]:
# Busiest routes (Q1 2019, excludes cancellations)
legs = flights_eng[valid_leg].copy()
by_route_dir = legs.groupby(['round_trip_id','leg_dir']).size().reset_index(name='legs')
rt_completed = by_route_dir.groupby('round_trip_id')['legs'].min().rename('completed_round_trips')
busiest = rt_completed.sort_values(ascending=False).head(10).reset_index()
busiest


,round_trip_id,completed_round_trips
0,LAX-SFO,4164
1,LGA-ORD,3576
2,LAS-LAX,3254
3,JFK-LAX,3158
4,LAX-SEA,2497
5,BOS-LGA,2405
6,HNL-OGG,2395
7,PDX-SEA,2376
8,ATL-MCO,2351
9,ATL-LGA,2293


In [7]:
# Revenue and operating profit (Q1 2019)
tix_q1 = tickets.copy()
if 'ROUNDTRIP' in tix_q1.columns:
    tix_q1 = tix_q1[tix_q1['ROUNDTRIP']==1]
tix_q1['round_trip_id'] = (
    tix_q1[['ORIGIN','DEST']].astype(str).stack().groupby(level=0).apply(lambda s: '-'.join(sorted(s.str.upper().str.strip())))
)
tix_q1['itinerary_revenue'] = tix_q1.get('PASSENGERS',0) * tix_q1.get('ITIN_FARE',0.0)
route_revenue = tix_q1.groupby('round_trip_id', as_index=False)['itinerary_revenue'].sum().rename(columns={'itinerary_revenue':'total_revenue'})
route_revenue.sort_values('total_revenue', ascending=False).head(10)


,round_trip_id,total_revenue
19276,JFK-LAX,3644415.0
13841,EWR-SFO,2039783.0
19351,JFK-SFO,1972276.0
13745,EWR-LAX,1534358.0
20569,LGA-ORD,1370380.0
2549,ATL-LAX,1327002.0
20054,LAX-ORD,1294862.0
2554,ATL-LGA,1255511.0
5747,BOS-SFO,1188451.0
13764,EWR-MCO,1183705.0


In [8]:
# Integrated route summary
summary = rt_completed.rename_axis('round_trip_id').reset_index(name='completed_round_trips')
punct = legs.groupby('round_trip_id')['on_time_arr'].mean().rename('on_time_arr_rate').reset_index()
summary = summary.merge(punct, on='round_trip_id', how='left')
summary = summary.merge(route_revenue, on='round_trip_id', how='left')
summary['operating_cost'] = 0.0
summary['operating_profit'] = summary['total_revenue'].fillna(0.0) - summary['operating_cost']
write_processed(summary, os.path.join(PROC,'route_summary.csv'))
summary.sort_values('operating_profit', ascending=False).head(10)


,round_trip_id,completed_round_trips,on_time_arr_rate,total_revenue,operating_cost,operating_profit
2098,JFK-LAX,3158,0.684494,3644415.0,0.0,3644415.0
1638,EWR-SFO,1212,0.550743,2039783.0,0.0,2039783.0
2126,JFK-SFO,1860,0.606557,1972276.0,0.0,1972276.0
1601,EWR-LAX,1056,0.660360,1534358.0,0.0,1534358.0
2298,LGA-ORD,3576,0.519424,1370380.0,0.0,1370380.0
189,ATL-LAX,1531,0.565952,1327002.0,0.0,1327002.0
2235,LAX-ORD,1857,0.608988,1294862.0,0.0,1294862.0
192,ATL-LGA,2293,0.624946,1255511.0,0.0,1255511.0
596,BOS-SFO,1128,0.589539,1188451.0,0.0,1188451.0
1604,EWR-MCO,1681,0.588043,1183705.0,0.0,1183705.0


In [9]:
# Recommendations (top five)
sc = summary.copy()
for col in ['operating_profit','completed_round_trips','on_time_arr_rate']:
    std = sc[col].std(ddof=0)
    sc[col+'_z'] = (sc[col]-sc[col].mean())/std if std and std>0 else 0
sc['score'] = 0.5*sc['operating_profit_z'] + 0.3*sc['completed_round_trips_z'] + 0.2*sc['on_time_arr_rate_z']
top5 = sc.sort_values('score', ascending=False).head(5)
top5[['round_trip_id','score','operating_profit','completed_round_trips','on_time_arr_rate']]


,round_trip_id,score,operating_profit,completed_round_trips,on_time_arr_rate
2098,JFK-LAX,11.547738,3644415.0,3158,0.684494
2126,JFK-SFO,5.993882,1972276.0,1860,0.606557
2298,LGA-ORD,5.706624,1370380.0,3576,0.519424
1638,EWR-SFO,5.525317,2039783.0,1212,0.550743
2254,LAX-SFO,5.363867,1020127.0,4164,0.559233


In [10]:
# Breakeven sizing (illustrative; aircraft purchase excluded)
aircraft_cost = 0  # set if provided externally
top5 = top5.copy()
top5['profit_per_round_trip'] = (top5['operating_profit'] / top5['completed_round_trips'].replace(0, pd.NA)).fillna(0.0)
top5['round_trips_to_breakeven'] = (aircraft_cost / top5['profit_per_round_trip']).replace([pd.NA, pd.NaT, float('inf')], 0)
top5[['round_trip_id','profit_per_round_trip','round_trips_to_breakeven']]


,round_trip_id,profit_per_round_trip,round_trips_to_breakeven
2098,JFK-LAX,1154.026282,0.0
2126,JFK-SFO,1060.363441,0.0
2298,LGA-ORD,383.215884,0.0
1638,EWR-SFO,1682.989274,0.0
2254,LAX-SFO,244.987272,0.0


In [11]:
# Charts (saved to docs/)
import plotly.express as px

# busiest
busiest_plot = summary.sort_values('completed_round_trips', ascending=False).head(10)
fig1 = px.bar(busiest_plot, x='round_trip_id', y='completed_round_trips', title='Top 10 busiest round-trip routes — Q1 2019 (excludes cancellations)')
fig1.update_layout(xaxis_tickangle=-45)
fig1.write_html(os.path.join(DOCS, 'busiest_routes.html'), include_plotlyjs='cdn')

# most profitable
prof_plot = summary.sort_values('operating_profit', ascending=False).head(10)
fig2 = px.bar(prof_plot, x='round_trip_id', y='operating_profit', title='Top 10 most profitable routes — Q1 2019 (excludes aircraft purchase)')
fig2.update_layout(xaxis_tickangle=-45)
fig2.write_html(os.path.join(DOCS, 'most_profitable_routes.html'), include_plotlyjs='cdn')

# profit vs on-time
fig3 = px.scatter(summary, x='on_time_arr_rate', y='operating_profit', size='completed_round_trips', hover_name='round_trip_id', title='Profit vs On-time arrival rate — Q1 2019 (excludes cancellations)')
fig3.write_html(os.path.join(DOCS, 'profit_vs_on_time.html'), include_plotlyjs='cdn')
'Charts written to docs/'


'Charts written to docs/'